In [118]:
import catboost
import pandas as pd
import numpy as np

In [119]:
df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
target = df['target']
test_target = test_df['target']
df.drop('target', axis=1, inplace=True)
test_df.drop('target', axis=1, inplace=True)

In [120]:
df.tail(100)

,total_pucrhases,average_psum,client_id,product_id,count,count_n,last_transaction,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol,std,mean,len
714912,7,1554.068571,9997,c790373926,1,0.142857,0.12,e344ab2e71,1d2939ba1d,da13a5d82a,c6789438a4,148,f431a4dab4,c4e167b91e,0.050,0,0,19.022224,71.975225,1776.0
714913,7,1554.068571,9997,c85e209e7d,1,0.142857,0.32,c3d3a8e8c6,034aca0659,453baf42c5,3b70cef8a8,281,cab440afaf,e6af81215a,0.100,0,0,10.567824,39.043459,55731.0
714914,7,1554.068571,9997,c92602bea7,1,0.142857,0.48,e344ab2e71,1d2939ba1d,831f318338,623f13717c,148,8790bebce1,8a09ca0fc7,0.138,0,0,21.920606,89.837582,5923.0
714915,7,1554.068571,9997,c9717e4bd1,1,0.142857,0.24,e344ab2e71,52f13dac0c,38816369ce,4d4b7e1f16,105,f7cced00f2,2445b69be2,0.400,0,0,15.347136,36.311420,24703.0
714916,7,1554.068571,9997,cc6e285908,1,0.142857,0.84,e344ab2e71,ed2ad1797c,6fbda2e7e2,13ad198857,187,f2b96aa11a,c4e167b91e,0.036,0,0,24.855853,71.165138,654.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715007,22,264.772727,9998,ecd6551517,1,0.045455,0.20,e344ab2e71,14d373dff5,1908d94f93,10fb31cfae,163,923bfb0b9b,91fab27374,0.330,0,0,48.073077,54.384044,4174.0
715008,22,264.772727,9998,ef2cdccd87,3,0.136364,0.30,e344ab2e71,ed2ad1797c,1d7d3cc9e8,aeb0ead42d,51,c4299d1b8a,fde6d50e35,0.105,0,0,44.290963,51.496503,3718.0
715009,22,264.772727,9998,f3534d8564,1,0.045455,0.10,e344ab2e71,703f4b6eb0,f4613d272f,47fc199714,9,85a5d05453,f8c5ecbd26,0.200,0,0,83.107695,132.111892,14800.0
715010,22,264.772727,9998,f60efc77d1,3,0.136364,0.60,c3d3a8e8c6,ad2b2e17d2,d7ca614c25,b201a329b9,214,ab230258e9,43acd80c1a,0.180,1,0,48.725106,91.404494,24611.0


In [121]:
client_id_map = {client_id: i for i, client_id in enumerate(df['client_id'].unique())}
tclient_id_map = {client_id: i for i, client_id in enumerate(test_df['client_id'].unique())}

train_groups = df['client_id'].map(client_id_map).values
test_groups = test_df['client_id'].map(tclient_id_map).values

target = np.array(target).astype(int)
test_target = np.array(test_target).astype(int)


cat_features = ['product_id', 'level_1', 'level_2', 'level_3', 'level_4', 'segment_id', 'brand_id', 'vendor_id']

In [122]:
cols = df.columns[:2].tolist() + df.columns[3:-1].tolist()

In [123]:
cols

['total_pucrhases',
 'average_psum',
 'product_id',
 'count',
 'count_n',
 'last_transaction',
 'level_1',
 'level_2',
 'level_3',
 'level_4',
 'segment_id',
 'brand_id',
 'vendor_id',
 'netto',
 'is_own_trademark',
 'is_alcohol',
 'std',
 'mean']

In [141]:
train_pool = catboost.Pool(df[cols], target, cat_features=cat_features, group_id=train_groups)

test_pool = catboost.Pool(test_df[cols], test_target, cat_features=cat_features, group_id=test_groups)

In [142]:
cb = catboost.CatBoostClassifier(cat_features=cat_features, eval_metric='MAP')
cb.fit(train_pool, eval_set=test_pool, early_stopping_rounds=30, )

Learning rate set to 0.15558
0:	learn: 0.2310053	test: 0.2371739	best: 0.2371739 (0)	total: 674ms	remaining: 11m 12s
1:	learn: 0.2594462	test: 0.2644417	best: 0.2644417 (1)	total: 1.18s	remaining: 9m 47s
2:	learn: 0.2673624	test: 0.2725718	best: 0.2725718 (2)	total: 1.9s	remaining: 10m 30s
3:	learn: 0.2686914	test: 0.2732281	best: 0.2732281 (3)	total: 2.51s	remaining: 10m 26s
4:	learn: 0.2828873	test: 0.2863152	best: 0.2863152 (4)	total: 3.14s	remaining: 10m 25s
5:	learn: 0.2886407	test: 0.2922724	best: 0.2922724 (5)	total: 3.79s	remaining: 10m 27s
6:	learn: 0.2913499	test: 0.2947263	best: 0.2947263 (6)	total: 4.39s	remaining: 10m 23s
7:	learn: 0.2937072	test: 0.2983544	best: 0.2983544 (7)	total: 5s	remaining: 10m 20s
8:	learn: 0.2938261	test: 0.2985028	best: 0.2985028 (8)	total: 5.57s	remaining: 10m 13s
9:	learn: 0.2943905	test: 0.2988539	best: 0.2988539 (9)	total: 6.18s	remaining: 10m 11s
10:	learn: 0.2952494	test: 0.2998428	best: 0.2998428 (10)	total: 6.7s	remaining: 10m 2s
11:	lear

92:	learn: 0.3084091	test: 0.3103581	best: 0.3105131 (90)	total: 58.9s	remaining: 9m 34s
93:	learn: 0.3085603	test: 0.3104070	best: 0.3105131 (90)	total: 59.5s	remaining: 9m 33s
94:	learn: 0.3084534	test: 0.3103244	best: 0.3105131 (90)	total: 1m	remaining: 9m 33s
95:	learn: 0.3084946	test: 0.3103229	best: 0.3105131 (90)	total: 1m	remaining: 9m 32s
96:	learn: 0.3085814	test: 0.3102854	best: 0.3105131 (90)	total: 1m 1s	remaining: 9m 31s
97:	learn: 0.3086493	test: 0.3103899	best: 0.3105131 (90)	total: 1m 2s	remaining: 9m 31s
98:	learn: 0.3090036	test: 0.3102646	best: 0.3105131 (90)	total: 1m 2s	remaining: 9m 30s
99:	learn: 0.3089558	test: 0.3103027	best: 0.3105131 (90)	total: 1m 3s	remaining: 9m 30s
100:	learn: 0.3090431	test: 0.3102810	best: 0.3105131 (90)	total: 1m 4s	remaining: 9m 29s
101:	learn: 0.3090247	test: 0.3102860	best: 0.3105131 (90)	total: 1m 4s	remaining: 9m 29s
102:	learn: 0.3091253	test: 0.3102962	best: 0.3105131 (90)	total: 1m 5s	remaining: 9m 29s
103:	learn: 0.3092348	te

182:	learn: 0.3132022	test: 0.3107224	best: 0.3110406 (152)	total: 2m	remaining: 8m 56s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.3110405874
bestIteration = 152

Shrink model to first 153 iterations.


In [133]:
cols

['total_pucrhases',
 'average_psum',
 'product_id',
 'count',
 'count_n',
 'last_transaction',
 'level_1',
 'level_2',
 'level_3',
 'level_4',
 'segment_id',
 'brand_id',
 'vendor_id',
 'netto',
 'is_own_trademark',
 'is_alcohol',
 'std',
 'mean']

In [134]:
pd.DataFrame(zip(cols, cb.get_feature_importance(test_pool)))

,0,1
0,total_pucrhases,9.709526
1,average_psum,2.401545
2,product_id,5.878857
3,count,26.597486
4,count_n,21.699308
5,last_transaction,1.075546
6,level_1,4.066872
7,level_2,2.343684
8,level_3,2.234125
9,level_4,1.112353


In [135]:
cb.save_model('../solution/models/catboost_rank.cb')

In [136]:
test_df['score'] = cb.predict_proba(test_df[cols])[:, 1]

test_df['target'] = test_target

scoring  = test_df[['client_id', 'score', 'target']].sort_values(['client_id', 'score'], ascending=[True, False])

scoring['rank'] = scoring.groupby('client_id').cumcount() + 1

scoring['cum_target'] = scoring.groupby('client_id')['target'].cumsum()

scoring['max_target'] = scoring.groupby('client_id')['target'].transform(sum)

In [137]:
test_df.head()

,total_pucrhases,average_psum,client_id,product_id,count,count_n,last_transaction,level_1,level_2,level_3,...,brand_id,vendor_id,netto,is_own_trademark,is_alcohol,std,mean,len,score,target
0,47,244.080426,0,00ebbaff99,1,0.021277,0.0,e344ab2e71,52f13dac0c,0f84eb7480,...,acd3dd483f,10486c3cf0,0.150,0,0,7.465743,28.791667,24.0,0.008927,0
1,47,244.080426,0,0439905bb4,2,0.042553,0.1,c3d3a8e8c6,f2333c90fb,419bc5b424,...,f3a4a72359,9cba311cbf,1.000,0,0,35.994717,174.452381,42.0,0.015703,0
2,47,244.080426,0,08b41f51e5,1,0.021277,0.2,e344ab2e71,52f13dac0c,a1786084ca,...,da50a90399,c9ab745e95,0.088,0,0,18.639348,36.139868,7264.0,0.007204,0
3,47,244.080426,0,0a3a44d22a,1,0.021277,0.2,e344ab2e71,14d373dff5,1908d94f93,...,923bfb0b9b,91fab27374,1.000,0,0,24.234923,64.037445,4807.0,0.007000,0
4,47,244.080426,0,0afbf1bdc2,1,0.021277,0.3,e344ab2e71,703f4b6eb0,8a37c27a14,...,e843772931,06eff35626,0.180,0,0,40.569433,70.579715,6448.0,0.005929,0


In [138]:
scoring['prec'] = ((scoring['rank'] <= 30) * scoring['target'] * (scoring['cum_target'] / scoring['rank']) / scoring['max_target'])

In [139]:
scoring.groupby('client_id').prec.sum().fillna(0).mean()

0.30086460483581273

In [58]:
test_df.head(100).style

,client_id,product_id,count,count_n,last_transaction,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol,std,mean,len,score,target
0,0,016225d8ff,1,0.024390,36,c3d3a8e8c6,034aca0659,b67737054d,8b5f0d5b1a,211,4da2dc345f,e6af81215a,1.000000,0,0,85.549524,121.161106,7343.000000,-4.346556,0
1,0,03d24b7e31,1,0.024390,35,e344ab2e71,14d373dff5,39532a0f6f,16341715da,92,5301d6b156,f4991c8355,0.450000,0,0,122.230194,120.010485,7630.000000,-4.670808,0
2,0,0434baefd8,1,0.024390,35,c3d3a8e8c6,ad2b2e17d2,eda7b2976b,d8172208f8,177,c19e6ad928,1fd329690d,0.090000,0,0,31.350275,53.955047,19843.000000,-4.212178,0
3,0,0446066c06,2,0.048780,33,e344ab2e71,ed2ad1797c,1f2ed5f7a5,a69b66110a,191,69ae2b3787,52618b6333,1.000000,0,0,48.007577,88.852838,44135.000000,-3.726499,0
4,0,050135fd94,1,0.024390,35,e344ab2e71,1d2939ba1d,396463b68f,044e47cd7f,118,57ea7d74b6,d3a9611996,0.934000,0,0,31.044195,103.715316,5269.000000,-4.699061,0
5,0,051de7c616,1,0.024390,7,c3d3a8e8c6,ad2b2e17d2,e33cc0b2a4,d3c4073308,1,f71e1a027b,af228f5fbe,0.975000,0,0,90.687825,93.039950,22453.000000,-4.673163,0
6,0,057fd570c6,3,0.073171,23,c3d3a8e8c6,ad2b2e17d2,e33cc0b2a4,5330a84194,1,062b7e85cd,4f276b13c1,1.400000,0,0,38.251967,102.970541,23490.000000,-3.056835,0
7,0,06ab962be9,1,0.024390,38,c3d3a8e8c6,d283080a93,f976a3c5f9,38c655d9b4,48,30e969f4cc,eefa3025e5,0.550000,0,0,71.178963,164.211986,1118.000000,-4.706683,0
8,0,080ace8748,2,0.048780,37,c3d3a8e8c6,ad2b2e17d2,2268617da5,7e34d7a162,320,8281de6bcb,6bc8b3c476,0.749000,0,0,39.403794,78.284015,166766.000000,-3.198518,0
9,0,087f9acb24,1,0.024390,34,c3d3a8e8c6,de6f3b925a,68b1fb172a,9b5cb21dca,0,37fd7fe235,43acd80c1a,0.300000,1,0,27.241182,116.474159,3657.000000,-4.130821,0
